In [1]:

! pip3 install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 MB 38.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 KB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 86.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 132.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.9/748.9 KB 83.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 79.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 178.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 KB 

In [2]:
import sys
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import numpy as np
# from multiprocessing import Pool  # , Process
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
import pandas as pd
import tensorflow as tf
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

2023-01-08 05:08:56.044009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 05:08:58.859284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-08 05:08:58.859341: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [2]:

activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 10 # [1, 100]
batch_size = 100
maximum_levels = 6 # [3,7]
maximum_units_per_level = 4 # [2,10]
maximum_neurons_per_unit = 4 # [2,20]
 





In [3]:

# Replace the data file with your data 
# and tell us what column you want this to predict

data_file = 'ames.csv'
prediction_target_column = 'price'

###### Ignore everything below this

raw_data = pd.read_csv(data_file)
needed_cols = [
    col for col in raw_data.columns if raw_data[col].dtype != 'object']
data_numeric = raw_data[needed_cols].fillna(0).astype(float)
label = raw_data.pop(prediction_target_column)

data_np = data_numeric.values

# np.save('label', label, allow_pickle=True)
# np.save('data', data_np, allow_pickle=True)

TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test'

# data_np = np.load('data.npy', allow_pickle=True)
# labels = np.load('labels.npy', allow_pickle=True)

tensor_x =\
    tf.constant(data_np)

training_x = [tensor_x]

INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]


#alcohol_labels = tf.constant(pd.get_dummies(pd.cut(white['alcohol'], np.arange(
#    white['alcohol'].min(), white['alcohol'].max()))).values)
# quality_labels = tf.constant(pd.get_dummies(white['quality']).values)

train_labels = [label.values]

# OUTPUT_SHAPES = [1]  # [train_labels[i].shape[1]
# for i in np.arange(len(train_labels))]


In [4]:


INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]

OUTPUT_SHAPES = [1]

meta_trial_number = str(int(np.random.random() * 10 ** 12))


cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.35,
    direction='minimize',
    metric_to_rank_by='val_root_mean_squared_error',
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation=None,
    number_of_architecture_moities_to_try=7,
    number_of_tries_per_architecture_moity=1,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    # use_multiprocessing_for_multiple_neural_networks=False,  # pull this param
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number)
val_root_mean_squared_error = cerebros_automl.run_random_search()


nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'1': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'4': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}]
3
Trying to create level 3
We think level 3's predecessors are: [0, 1, 2]
k is: 4 value is: [{'3': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}]
4
Trying to create level 4
We think level 4's predecessors are: [0, 1, 2, 3]
k is: 5 value is: [{'4': <class 'cerebros.units.units.DenseU

returning trial 0 oracles
           loss  root_mean_squared_error      val_loss  \
0  4.089696e+10            202229.968750  3.578870e+10   
1  4.089642e+10            202228.625000  3.578977e+10   
2  4.089591e+10            202227.375000  3.577428e+10   
3  4.089539e+10            202226.078125  3.576259e+10   
4  4.089477e+10            202224.546875  3.576291e+10   
5  4.089411e+10            202222.921875  3.574890e+10   
6  4.089335e+10            202221.046875  3.573870e+10   
7  4.089254e+10            202219.046875  3.573626e+10   
8  4.089157e+10            202216.640625  3.572942e+10   
9  4.089051e+10            202214.015625  3.571951e+10   

   val_root_mean_squared_error  trial_number  subtrial_number  \
0                189179.015625             0                0   
1                189181.843750             0                0   
2                189140.906250             0                0   
3                189110.000000             0                0   
4         

returning trial 1 oracles
           loss  root_mean_squared_error      val_loss  \
0  4.089709e+10            197696.890625  3.569044e+10   
1  4.089674e+10            202229.421875  3.571361e+10   
2  4.089637e+10            202228.515625  3.572054e+10   
3  4.089596e+10            202227.500000  3.571915e+10   
4  4.089550e+10            202226.359375  3.571190e+10   
5  4.089494e+10            202224.968750  3.570392e+10   
6  4.089430e+10            202223.375000  3.569619e+10   
7  4.089354e+10            202221.515625  3.568750e+10   
8  4.089267e+10            202219.359375  3.567881e+10   
9  4.089172e+10            202217.000000  3.567501e+10   

   val_root_mean_squared_error  trial_number  subtrial_number  \
0                188919.125000             1                0   
1                188980.453125             1                0   
2                188998.765625             1                0   
3                188995.109375             1                0   
4         

returning trial 2 oracles
           loss  root_mean_squared_error      val_loss  \
0  4.089716e+10            197657.578125  3.558124e+10   
1  4.089676e+10            202229.484375  3.562736e+10   
2  4.089635e+10            202228.453125  3.565371e+10   
3  4.089589e+10            202227.312500  3.565775e+10   
4  4.089539e+10            202226.078125  3.565608e+10   
5  4.089479e+10            202224.593750  3.565500e+10   
6  4.089410e+10            202222.906250  3.565230e+10   
7  4.089333e+10            202220.984375  3.565058e+10   
8  4.089245e+10            202218.812500  3.564943e+10   
9  4.089146e+10            202216.359375  3.564846e+10   

   val_root_mean_squared_error  trial_number  subtrial_number  \
0                188629.906250             2                0   
1                188752.125000             2                0   
2                188821.906250             2                0   
3                188832.609375             2                0   
4         

returning trial 3 oracles
           loss  root_mean_squared_error      val_loss  \
0  4.089714e+10            197634.031250  3.563021e+10   
1  4.089688e+10            202229.781250  3.565263e+10   
2  4.089661e+10            202229.093750  3.566984e+10   
3  4.089631e+10            202228.359375  3.568262e+10   
4  4.089594e+10            202227.453125  3.568601e+10   
5  4.089553e+10            202226.437500  3.567994e+10   
6  4.089506e+10            202225.265625  3.567299e+10   
7  4.089452e+10            202223.921875  3.566604e+10   
8  4.089391e+10            202222.421875  3.566807e+10   
9  4.089324e+10            202220.765625  3.566634e+10   

   val_root_mean_squared_error  trial_number  subtrial_number  \
0                188759.656250             3                0   
1                188819.046875             3                0   
2                188864.593750             3                0   
3                188898.453125             3                0   
4         

returning trial 4 oracles
           loss  root_mean_squared_error      val_loss  \
0  4.089720e+10            197649.968750  3.573203e+10   
1  4.089687e+10            202229.734375  3.575975e+10   
2  4.089654e+10            202228.937500  3.577010e+10   
3  4.089618e+10            202228.031250  3.576781e+10   
4  4.089578e+10            202227.062500  3.575043e+10   
5  4.089533e+10            202225.937500  3.572972e+10   
6  4.089484e+10            202224.718750  3.571226e+10   
7  4.089424e+10            202223.250000  3.569711e+10   
8  4.089354e+10            202221.515625  3.570078e+10   
9  4.089276e+10            202219.578125  3.569763e+10   

   val_root_mean_squared_error  trial_number  subtrial_number  \
0                189029.187500             4                0   
1                189102.484375             4                0   
2                189129.843750             4                0   
3                189123.796875             4                0   
4         

returning trial 5 oracles
           loss  root_mean_squared_error      val_loss  \
0  4.089712e+10            197677.546875  3.547667e+10   
1  4.089672e+10            202229.390625  3.555647e+10   
2  4.089634e+10            202228.437500  3.560367e+10   
3  4.089592e+10            202227.406250  3.562723e+10   
4  4.089545e+10            202226.234375  3.563372e+10   
5  4.089493e+10            202224.953125  3.563447e+10   
6  4.089433e+10            202223.468750  3.563083e+10   
7  4.089369e+10            202221.875000  3.563376e+10   
8  4.089288e+10            202219.890625  3.563525e+10   
9  4.089206e+10            202217.859375  3.563869e+10   

   val_root_mean_squared_error  trial_number  subtrial_number  \
0                188352.515625             5                0   
1                188564.234375             5                0   
2                188689.343750             5                0   
3                188751.750000             5                0   
4         

returning trial 6 oracles
           loss  root_mean_squared_error      val_loss  \
0  4.089711e+10            197625.312500  3.584262e+10   
1  4.089684e+10            202229.687500  3.583065e+10   
2  4.089655e+10            202228.968750  3.580910e+10   
3  4.089622e+10            202228.140625  3.578959e+10   
4  4.089582e+10            202227.140625  3.577070e+10   
5  4.089533e+10            202225.937500  3.575936e+10   
6  4.089478e+10            202224.562500  3.575405e+10   
7  4.089415e+10            202223.031250  3.574718e+10   
8  4.089344e+10            202221.250000  3.574252e+10   
9  4.089262e+10            202219.250000  3.573461e+10   

   val_root_mean_squared_error  trial_number  subtrial_number  \
0                189321.468750             6                0   
1                189289.859375             6                0   
2                189232.906250             6                0   
3                189181.375000             6                0   
4         

In [5]:
print(val_root_mean_squared_error)

188352.515625
